In [2]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
from PIL import Image
import pickle
import shutil
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
from keras.regularizers import l2
from keras.activations import *
from DistanceLayer import DistanceLayer
from SiameseModel import SiameseModel
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)



# batch_size = 96
# batch_size = 8
batch_size = 24
# batch_size = 3
# batch_size = 1

2.8.0


In [3]:
features = {}
# load the features dictionary from the file
with open('features_merged.pickle', 'rb') as handle:
    features = pickle.load(handle)

feature_dim = features["02461"].shape[0]

# Model

a lot of the code comes from https://keras.io/examples/vision/siamese_network/
and https://github.com/akarshzingade/image-similarity-deep-ranking/blob/master/deepRanking.py

In [4]:


embedding = tf.keras.Sequential([
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
])




anchor_input = layers.Input(name="anchor", shape=(feature_dim))
positive_input = layers.Input(name="positive", shape=(feature_dim))
negative_input = layers.Input(name="negative", shape=(feature_dim))

distances = DistanceLayer()(
    embedding(anchor_input),
    embedding(positive_input),
    embedding(negative_input),
)

model = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

# model.summary()

In [5]:
#convert the features dictionary to a tensorflow hash table
keys = tf.constant([key for key in features.keys()])
values = tf.constant([val for val in features.values()])

table = tf.lookup.experimental.DenseHashTable(
    value_dtype=tf.float32,
    key_dtype=tf.string,
    empty_key="empty_key",
    deleted_key="deleted_key",
    default_value=[-1]*feature_dim,
    )

table.insert(keys, values)

In [6]:
#load strings from train_triplets.txt
train_triplets = np.loadtxt('./train_triplets.txt', dtype=str,delimiter = ' ')
dataset = tf.data.Dataset.from_tensor_slices(train_triplets)


@tf.function
def load_image(inputs):
    filenames = inputs
    anchor = table[filenames[0]]
    positive = table[filenames[1]]
    negative = table[filenames[2]]
    output = (anchor,positive,negative)
    
    return output

dataset = dataset.map(load_image).cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(20)

In [7]:
siamese_model = SiameseModel(model,margin=0.5)
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(0.000001))


In [8]:

#this is needed, otherwise nothing works
pred = siamese_model.predict(next(iter(dataset)))
print(pred)

(array([0.6016176 , 0.44964498, 0.61970204, 0.6067483 , 0.39319363,
       0.45117542, 0.5056751 , 0.57171255, 0.5471591 , 0.35098958,
       0.5197375 , 0.5225626 , 0.47206536, 0.49073398, 0.36871636,
       0.35955945, 0.5062956 , 0.4259107 , 0.3848334 , 0.55850756,
       0.41393223, 0.53892905, 0.5991969 , 0.45919362], dtype=float32), array([0.5375221 , 0.5191359 , 0.4925824 , 0.5406202 , 0.5117215 ,
       0.6171048 , 0.52930355, 0.4977427 , 0.5557702 , 0.5173627 ,
       0.6315845 , 0.5607286 , 0.57168365, 0.50169796, 0.5995406 ,
       0.5579188 , 0.57875454, 0.5034418 , 0.4823468 , 0.60522795,
       0.4631008 , 0.58768576, 0.6597943 , 0.56674516], dtype=float32))


In [9]:
train_steps_per_epoch = int((59520)/batch_size)
checkpoint_filepath = './checkpoints/checkpoint-{epoch}/'
#save checkpoint after every epoch
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
)


siamese_model.fit(dataset, epochs=100,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[model_checkpoint_callback])



Epoch 1/100
2480/2480 [==============================] - 59s 24ms/step - loss: 0.3288
Epoch 2/100
 848/2480 [=========>....................] - ETA: 32s - loss: 0.2812

KeyboardInterrupt: 

In [ ]:
# tf.keras.utils.plot_model(
#     model,
#     to_file="model.png",
#     show_shapes=True,
#     expand_nested=True,
    
# )